# Розділ 7: Створення чат-додатків
## Швидкий старт з Azure OpenAI API


## Огляд
Цей блокнот адаптовано з [репозиторію прикладів Azure OpenAI](https://github.com/Azure/azure-openai-samples?WT.mc_id=academic-105485-koreyst), який містить блокноти, що також отримують доступ до сервісу [OpenAI](notebook-openai.ipynb).

Python API OpenAI також працює з Azure OpenAI, з деякими модифікаціями. Дізнайтеся більше про відмінності тут: [Як переключатися між кінцевими точками OpenAI та Azure OpenAI за допомогою Python](https://learn.microsoft.com/azure/ai-services/openai/how-to/switching-endpoints?WT.mc_id=academic-109527-jasmineg)

Для отримання додаткових прикладів швидкого старту зверніться до офіційної [документації Azure OpenAI Quickstart](https://learn.microsoft.com/azure/cognitive-services/openai/quickstart?pivots=programming-language-studio&WT.mc_id=academic-105485-koreyst)


## Зміст  

[Огляд](../../../../07-building-chat-applications/python)  
[Початок роботи з Azure OpenAI Service](../../../../07-building-chat-applications/python)  
[Створіть свій перший запит](../../../../07-building-chat-applications/python)  

[Випадки використання](../../../../07-building-chat-applications/python)    
[1. Підсумувати текст](../../../../07-building-chat-applications/python)  
[2. Класифікувати текст](../../../../07-building-chat-applications/python)  
[3. Генерувати нові назви продуктів](../../../../07-building-chat-applications/python)  
[4. Тонке налаштування класифікатора](../../../../07-building-chat-applications/python)  
[5. Вбудовування](../../../../07-building-chat-applications/python)

[Посилання](../../../../07-building-chat-applications/python)


### Початок роботи з Azure OpenAI Service

Новим клієнтам потрібно [подати заявку на доступ](https://aka.ms/oai/access?WT.mc_id=academic-105485-koreyst) до Azure OpenAI Service.  
Після затвердження клієнти можуть увійти в портал Azure, створити ресурс Azure OpenAI Service і почати експериментувати з моделями через студію  

[Відмінний ресурс для швидкого початку](https://techcommunity.microsoft.com/blog/educatordeveloperblog/azure-openai-service-is-now-generally-available/3719177?WT.mc_id=academic-105485-koreyst)


### Створіть свій перший запит  
Ця коротка вправа надасть базове введення для надсилання запитів до моделі OpenAI для простої задачі "резюмування".


**Кроки**:  
1. Встановіть бібліотеку OpenAI у своєму середовищі Python  
2. Завантажте стандартні допоміжні бібліотеки та встановіть типові облікові дані безпеки OpenAI для служби OpenAI, яку ви створили  
3. Виберіть модель для вашого завдання  
4. Створіть простий запит для моделі  
5. Надішліть свій запит до API моделі!


### 1. Встановіть OpenAI


  > [!ПРИМІТКА] Цей крок не є обов’язковим, якщо запускати цей ноутбук у Codespaces або в межах Devcontainer


In [ ]:
%pip install openai python-dotenv

### 2. Імпортуйте допоміжні бібліотеки та створіть облікові дані


In [ ]:
import os
from openai import AzureOpenAI
import numpy as np
from dotenv import load_dotenv
load_dotenv()

#validate data inside .env file

client = AzureOpenAI(
  api_key=os.environ['AZURE_OPENAI_API_KEY'],  # this is also the default, it can be omitted
  api_version = "2023-05-15"
  )

### 3. Вибір правильної моделі  
Моделі GPT-3.5-turbo або GPT-4 можуть розуміти та генерувати природну мову. Сервіс пропонує чотири можливості моделей, кожна з яких має різні рівні потужності та швидкості, що підходять для різних завдань. 

[Azure OpenAI models](https://learn.microsoft.com/azure/cognitive-services/openai/concepts/models?WT.mc_id=academic-105485-koreyst)


In [ ]:
# Select the General Purpose curie model for text
model = os.environ['AZURE_OPENAI_DEPLOYMENT']

## 4. Проєктування підказок  

"Магія великих мовних моделей полягає в тому, що, навчаючись мінімізувати цю помилку прогнозування на величезних обсягах тексту, моделі в кінцевому підсумку вивчають корисні для цих прогнозів концепції. Наприклад, вони вивчають такі концепції"(1):

* як писати правильно
* як працює граматика
* як перефразовувати
* як відповідати на запитання
* як вести розмову
* як писати багатьма мовами
* як програмувати
* тощо.

#### Як керувати великою мовною моделлю  
"З усіх вхідних даних для великої мовної моделі найбільший вплив має текстова підказка(1).

Великі мовні моделі можна стимулювати до створення вихідних даних кількома способами:

- Інструкція: скажіть моделі, чого ви хочете
- Завершення: спонукайте модель завершити початок того, що ви хочете
- Демонстрація: покажіть моделі, чого ви хочете, за допомогою:
- Кількох прикладів у підказці
- Багатьох сотень або тисяч прикладів у навчальному наборі для тонкого налаштування



#### Існує три основні рекомендації щодо створення підказок:

**Показуйте і пояснюйте**. Чітко вказуйте, чого ви хочете, через інструкції, приклади або їх комбінацію. Якщо ви хочете, щоб модель впорядкувала список елементів в алфавітному порядку або класифікувала абзац за настроєм, покажіть їй, що саме ви хочете.

**Надавайте якісні дані**. Якщо ви намагаєтеся створити класифікатор або змусити модель слідувати певному шаблону, переконайтеся, що прикладів достатньо. Обов’язково перевіряйте свої приклади — модель зазвичай достатньо розумна, щоб помічати базові орфографічні помилки і давати відповідь, але вона також може припустити, що це зроблено навмисно, і це може вплинути на відповідь.

**Перевіряйте свої налаштування.** Параметри temperature і top_p контролюють, наскільки детермінованою є модель при генерації відповіді. Якщо ви просите відповідь, де є лише одна правильна відповідь, тоді варто встановити ці параметри нижчими. Якщо ви шукаєте більш різноманітні відповіді, тоді варто встановити їх вищими. Найпоширеніша помилка людей із цими налаштуваннями — вважати, що вони контролюють "кмітливість" або "креативність".


Джерело: https://learn.microsoft.com/azure/ai-services/openai/overview


зображення створює ваш перший текстовий запит!


### 5. Надіслати!


In [ ]:
# Create your first prompt
text_prompt = "Should oxford commas always be used?"

response = client.chat.completions.create(
  model=model,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":text_prompt},])

response.choices[0].message.content

### Повторіть той самий виклик, як порівнюються результати?


In [ ]:

response = client.chat.completions.create(
  model=model,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":text_prompt},])

response.choices[0].message.content

## Підсумуйте текст  
#### Виклик  
Підсумуйте текст, додавши «tl;dr:» в кінці текстового фрагмента. Зверніть увагу, як модель розуміє, як виконувати низку завдань без додаткових інструкцій. Ви можете експериментувати з більш описовими підказками, ніж tl;dr, щоб змінити поведінку моделі та налаштувати отримане резюме(3).  

Останні дослідження продемонстрували значні покращення у багатьох завданнях і бенчмарках NLP шляхом попереднього навчання на великому корпусі тексту з подальшим донавчанням на конкретному завданні. Хоча архітектура зазвичай є незалежною від завдання, цей метод все одно вимагає наборів даних для донавчання, специфічних для завдання, що містять тисячі або десятки тисяч прикладів. Натомість люди зазвичай можуть виконувати нове мовне завдання, маючи лише кілька прикладів або прості інструкції — те, з чим сучасні системи NLP досі значною мірою борються. Тут ми показуємо, що масштабування мовних моделей значно покращує продуктивність у режимі few-shot, незалежно від завдання, іноді навіть досягаючи конкурентоспроможності з попередніми передовими методами донавчання.  

Tl;dr


# Вправи для кількох випадків використання  
1. Підсумувати текст  
2. Класифікувати текст  
3. Генерувати нові назви продуктів  
4. Вбудовування  
5. Тонке налаштування класифікатора


In [ ]:
prompt = "Recent work has demonstrated substantial gains on many NLP tasks and benchmarks by pre-training on a large corpus of text followed by fine-tuning on a specific task. While typically task-agnostic in architecture, this method still requires task-specific fine-tuning datasets of thousands or tens of thousands of examples. By contrast, humans can generally perform a new language task from only a few examples or from simple instructions - something that current NLP systems still largely struggle to do. Here we show that scaling up language models greatly improves task-agnostic, few-shot performance, sometimes even reaching competitiveness with prior state-of-the-art fine-tuning approaches.\n\ntl;dr"


In [ ]:
#Setting a few additional, typical parameters during API Call

response = client.chat.completions.create(
  model=model,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":prompt},])

response.choices[0].message.content

## Класифікувати текст  
#### Виклик  
Класифікуйте елементи за категоріями, наданими під час інференсу. У наведеному нижче прикладі ми надаємо як категорії, так і текст для класифікації у запиті (*playground_reference). 

Запит клієнта: Привіт, нещодавно зламалася одна з клавіш на клавіатурі мого ноутбука, і мені потрібна заміна:

Класифікована категорія:


In [ ]:
prompt = "Classify the following inquiry into one of the following: categories: [Pricing, Hardware Support, Software Support]\n\ninquiry: Hello, one of the keys on my laptop keyboard broke recently and I'll need a replacement:\n\nClassified category:"
print(prompt)

In [ ]:
#Setting a few additional, typical parameters during API Call

response = client.chat.completions.create(
  model=model,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":prompt},])

response.choices[0].message.content

## Генерація нових назв продуктів
#### Виклик
Створіть назви продуктів на основі прикладів слів. Тут ми включаємо в підказку інформацію про продукт, для якого збираємося генерувати назви. Ми також надаємо подібний приклад, щоб показати бажаний шаблон. Крім того, ми встановили високе значення температури, щоб збільшити випадковість і отримати більш інноваційні відповіді.

Опис продукту: Домашній міксер для молочних коктейлів  
Слово-насіння: швидкий, здоровий, компактний.  
Назви продуктів: HomeShaker, Fit Shaker, QuickShake, Shake Maker

Опис продукту: Пара взуття, яка підходить для будь-якого розміру ноги.  
Слово-насіння: адаптивний, підходить, омні-фіт.


In [ ]:
prompt = "Product description: A home milkshake maker\nSeed words: fast, healthy, compact.\nProduct names: HomeShaker, Fit Shaker, QuickShake, Shake Maker\n\nProduct description: A pair of shoes that can fit any foot size.\nSeed words: adaptable, fit, omni-fit."

print(prompt)

In [ ]:
#Setting a few additional, typical parameters during API Call

response = client.chat.completions.create(
  model=model,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":prompt}])

response.choices[0].message.content

## Вбудовування
У цьому розділі показано, як отримувати вбудовування та знаходити схожість між словами, реченнями та документами. Щоб запустити наступні блокноти, потрібно розгорнути модель, яка використовує `text-embedding-ada-002` як базову модель, і встановити ім'я її розгортання у файлі .env, використовуючи змінну `AZURE_OPENAI_EMBEDDINGS_DEPLOYMENT`.


### Таксономія моделей - Вибір моделі вбудовування

**Таксономія моделей**: {family} - {capability} - {input-type} - {identifier}  

{family}     --> text-embedding  (сімейство вбудовувань)  
{capability} --> ada             (всі інші моделі вбудовування будуть виведені з експлуатації у 2024 році)  
{input-type} --> n/a             (вказується лише для моделей пошуку)  
{identifier} --> 002             (версія 002)  

model = 'text-embedding-ada-002'


  > [!ПРИМІТКА] Наступний крок не є обов’язковим, якщо запускати цей ноутбук у Codespaces або в межах Devcontainer


In [ ]:
# Dependencies for embeddings_utils
%pip install matplotlib plotly scikit-learn pandas

In [ ]:
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

In [ ]:
text = 'the quick brown fox jumped over the lazy dog'
model= os.environ['AZURE_OPENAI_EMBEDDINGS_DEPLOYMENT']
client.embeddings.create(input='[text]', model=model).data[0].embedding

In [ ]:

# compare several words
automobile_embedding  = client.embeddings.create(input='automobile', model=model).data[0].embedding
vehicle_embedding     = client.embeddings.create(input='vehicle', model=model).data[0].embedding
dinosaur_embedding    = client.embeddings.create(input='dinosaur', model=model).data[0].embedding
stick_embedding       = client.embeddings.create(input='stick', model=model).data[0].embedding

print(cosine_similarity(automobile_embedding, vehicle_embedding))
print(cosine_similarity(automobile_embedding, dinosaur_embedding))
print(cosine_similarity(automobile_embedding, stick_embedding))

## Порівняння статті з набору даних cnn daily news
джерело: https://huggingface.co/datasets/cnn_dailymail


In [ ]:
import pandas as pd
cnn_daily_articles = ['BREMEN, Germany -- Carlos Alberto, who scored in FC Porto\'s Champions League final victory against Monaco in 2004, has joined Bundesliga club Werder Bremen for a club record fee of 7.8 million euros ($10.7 million). Carlos Alberto enjoyed success at FC Porto under Jose Mourinho. "I\'m here to win titles with Werder," the 22-year-old said after his first training session with his new club. "I like Bremen and would only have wanted to come here." Carlos Alberto started his career with Fluminense, and helped them to lift the Campeonato Carioca in 2002. In January 2004 he moved on to FC Porto, who were coached by José Mourinho, and the club won the Portuguese title as well as the Champions League. Early in 2005, he moved to Corinthians, where he impressed as they won the Brasileirão,but in 2006 Corinthians had a poor season and Carlos Alberto found himself at odds with manager, Emerson Leão. Their poor relationship came to a climax at a Copa Sul-Americana game against Club Atlético Lanús, and Carlos Alberto declared that he would not play for Corinthians again while Leão remained as manager. Since January this year he has been on loan with his first club Fluminense. Bundesliga champions VfB Stuttgart said on Sunday that they would sign a loan agreement with Real Zaragoza on Monday for Ewerthon, the third top Brazilian player to join the German league in three days. A VfB spokesman said Ewerthon, who played in the Bundesliga for Borussia Dortmund from 2001 to 2005, was expected to join the club for their pre-season training in Austria on Monday. On Friday, Ailton returned to Germany where he was the league\'s top scorer in 2004, signing a one-year deal with Duisburg on a transfer from Red Star Belgrade. E-mail to a friend .',
                        '(CNN) -- Football superstar, celebrity, fashion icon, multimillion-dollar heartthrob. Now, David Beckham is headed for the Hollywood Hills as he takes his game to U.S. Major League Soccer. CNN looks at how Bekham fulfilled his dream of playing for Manchester United, and his time playing for England. The world\'s famous footballer has begun a five-year contract with the Los Angeles Galaxy team, and on Friday Beckham will meet the press and reveal his new shirt number. This week, we take an in depth look at the life and times of Beckham, as CNN\'s very own "Becks," Becky Anderson, sets out to examine what makes the man tick -- as footballer, fashion icon and global phenomenon. It\'s a long way from the streets of east London to the Hollywood Hills and Becky charts Beckham\'s incredible rise to football stardom, a journey that has seen his skills grace the greatest stages in world soccer. She goes in pursuit of the current hottest property on the sports/celebrity circuit in the U.S. and along the way explores exactly what\'s behind the man with the golden boot. CNN will look back at the life of Beckham, the wonderfully talented youngster who fulfilled his dream of playing for Manchester United, his marriage to pop star Victoria, and the trials and tribulations of playing for England. We\'ll look at the highs (scoring against Greece), the lows (being sent off during the World Cup), the Man. U departure for the Galacticos of Madrid -- and now the Home Depot stadium in L.A. We\'ll ask how Beckham and his family will adapt to life in Los Angeles -- the people, the places to see and be seen and the celebrity endorsement. Beckham is no stranger to exposure. He has teamed with Reggie Bush in an Adidas commercial, is the face of Motorola, is the face on a PlayStation game and doesn\'t need fashion tips as he has his own international clothing line. But what does the star couple need to do to become an accepted part of Tinseltown\'s glitterati? The road to major league football in the U.S.A. is a well-worn route for some of the world\'s greatest players. We talk to some of the former greats who came before him and examine what impact these overseas stars had on U.S. soccer and look at what is different now. We also get a rare glimpse inside the David Beckham academy in L.A, find out what drives the kids and who are their heroes. The perception that in the U.S.A. soccer is a "game for girls" after the teenage years is changing. More and more young kids are choosing the European game over the traditional U.S. sports. E-mail to a friend .',
                        'LOS ANGELES, California (CNN) -- Youssif, the 5-year-old burned Iraqi boy, rounded the corner at Universal Studios when suddenly the little boy hero met his favorite superhero. Youssif has always been a huge Spider-Man fan. Meeting him was "my favorite thing," he said. Spider-Man was right smack dab in front of him, riding a four-wheeler amid a convoy of other superheroes. The legendary climber of buildings and fighter of evil dismounted, walked over to Youssif and introduced himself. Spidey then gave the boy from a far-away land a gentle hug, embracing him in his iconic blue and red tights. He showed Youssif a few tricks, like how to shoot a web from his wrist. Only this time, no web was spun. "All right Youssif!" Spider-Man said after the boy mimicked his wrist movement. Other superheroes crowded around to get a closer look. Even the Green Goblin stopped his villainous ways to tell the boy hi. Youssif remained unfazed. He didn\'t take a liking to Spider-Man\'s nemesis. Spidey was just too cool. "It was my favorite thing," the boy said later. "I want to see him again." He then felt compelled to add: "I know it\'s not the real Spider-Man." This was the day of dreams when the boy\'s nightmares were, at least temporarily, forgotten. He met SpongeBob, Lassie and a 3-year-old orangutan named Archie. The hairy, brownish-red primate took to the boy, grabbing his hand and holding it. Even when Youssif pulled away, Archie would inch his hand back toward the boy\'s and then snatch it. See Youssif enjoy being a boy again » . The boy giggled inside a play area where sponge-like balls shot out of toy guns. It was a far different artillery than what he was used to seeing in central Baghdad, as recently as a week ago. He squealed with delight and raced around the room collecting as many balls as he could. He rode a tram through the back stages at Universal Studios. At one point, the car shook. Fire and smoke filled the air, debris cascaded down and a big rig skidded toward the vehicle. The boy and his family survived the pretend earthquake unscathed. "Even I was scared," the dad said. "Well, I wasn\'t," Youssif replied. The father and mother grinned from ear to ear throughout the day. Youssif pushed his 14-month-old sister, Ayaa, in a stroller. "Did you even need to ask us if we were interested in coming here?" Youssif\'s father said in amazement. "Other than my wedding day, this is the happiest day of my life," he said. Just a day earlier, the mother and father talked about their journey out of Iraq and to the United States. They also discussed that day nine months ago when masked men grabbed their son outside the family home, doused him in gas and set him on fire. His mother heard her boy screaming from inside. The father sought help for his boy across Baghdad, but no one listened. He remembers his son\'s two months of hospitalization. The doctors didn\'t use anesthetics. He could hear his boy\'s piercing screams from the other side of the hospital. Watch Youssif meet his doctor and play with his little sister » . The father knew that speaking to CNN would put his family\'s lives in jeopardy. The possibility of being killed was better than seeing his son suffer, he said. "Anything for Youssif," he said. "We had to do it." They described a life of utter chaos in Baghdad. Neighbors had recently given birth to a baby girl. Shortly afterward, the father was kidnapped and killed. Then, there was the time when some girls wore tanktops and jeans. They were snatched off the street by gunmen. The stories can be even more gruesome. The couple said they had heard reports that a young girl was kidnapped and beheaded --and her killers sewed a dog\'s head on the corpse and delivered it to her family\'s doorstep. "These are just some of the stories," said Youssif\'s mother, Zainab. Under Saddam Hussein, there was more security and stability, they said. There was running water and electricity most of the time. But still life was tough under the dictator, like the time when Zainab\'s uncle disappeared and was never heard from again after he read a "religious book," she said. Sitting in the parking lot of a Target in suburban Los Angeles, Youssif\'s father watched as husbands and wives, boyfriends and girlfriends, parents and their children, came and went. Some held hands. Others smiled and laughed. "Iraq finished," he said in what few English words he knows. He elaborated in Arabic: His homeland won\'t be enjoying such freedoms anytime soon. It\'s just not possible. Too much violence. Too many killings. His two children have only seen war. But this week, the family has seen a much different side of America -- an outpouring of generosity and a peaceful nation at home. "It\'s been a dream," the father said. He used to do a lot of volunteer work back in Baghdad. "Maybe that\'s why I\'m being helped now," the father said. At Universal Studios, he looked out across the valley below. The sun glistened off treetops and buildings. It was a picturesque sight fit for a Hollywood movie. "Good America, good America," he said in English. E-mail to a friend . CNN\'s Arwa Damon contributed to this report.'
]

cnn_daily_article_highlights = ['Werder Bremen pay a club record $10.7 million for Carlos Alberto .\nThe Brazilian midfielder won the Champions League with FC Porto in 2004 .\nSince January he has been on loan with his first club, Fluminense .',
                                'Beckham has agreed to a five-year contract with Los Angeles Galaxy .\nNew contract took effect July 1, 2007 .\nFormer English captain to meet press, unveil new shirt number Friday .\nCNN to look at Beckham as footballer, fashion icon and global phenomenon .',
                                'Boy on meeting Spider-Man: "It was my favorite thing"\nYoussif also met SpongeBob, Lassie and an orangutan at Universal Studios .\nDad: "Other than my wedding day, this is the happiest day of my life"'
]

cnn_df = pd.DataFrame({"articles":cnn_daily_articles, "highligths":cnn_daily_article_highlights})

cnn_df.head()

In [ ]:
article1_embedding    = client.embeddings.create(input=cnn_df.articles.iloc[0], model=model).data[0].embedding
article2_embedding    = client.embeddings.create(input=cnn_df.articles.iloc[1], model=model).data[0].embedding
article3_embedding    = client.embeddings.create(input=cnn_df.articles.iloc[2], model=model).data[0].embedding

print(cosine_similarity(article1_embedding, article2_embedding))
print(cosine_similarity(article1_embedding, article3_embedding))

# References  
- [Azure Documentation - Azure OpenAI Models](https://learn.microsoft.com/azure/cognitive-services/openai/concepts/models?WT.mc_id=academic-105485-koreyst)  
- [OpenAI Studio Examples](https://oai.azure.com/portal?WT.mc_id=academic-105485-koreyst)  


# Для додаткової допомоги  
[OpenAI Commercialization Team](AzureOpenAITeam@microsoft.com)


# Учасники
* Louis Li  


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Відмова від відповідальності**:  
Цей документ було перекладено за допомогою сервісу автоматичного перекладу [Co-op Translator](https://github.com/Azure/co-op-translator). Хоча ми прагнемо до точності, будь ласка, майте на увазі, що автоматичні переклади можуть містити помилки або неточності. Оригінальний документ рідною мовою слід вважати авторитетним джерелом. Для критично важливої інформації рекомендується звертатися до професійного людського перекладу. Ми не несемо відповідальності за будь-які непорозуміння або неправильні тлумачення, що виникли внаслідок використання цього перекладу.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
